In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Styling
plt.style.use('tdrstyle.mplstyle')

In [2]:
from rootpy.io import root_open
import ROOT
ROOT.gROOT.SetBatch(True)

Welcome to JupyROOT 6.10/09


In [3]:
infile_r = None  # input file handle

def load_pgun():
  global infile_r
  infile = '../test7/ntuple_SingleMuon_Toy_2GeV_add.6.root'
  #if use_condor:
  #  infile = 'root://cmsio5.rc.ufl.edu//store/user/jiafulow/L1MuonTrigger/P2_10_1_5/SingleMuon_Toy_2GeV/'+infile
  infile_r = root_open(infile)
  tree = infile_r.ntupler.tree
  #tree = TreeChain('ntupler/tree', [infile])
  print('[INFO] Opening file: %s' % infile)

  # Define collection
  tree.define_collection(name='hits', prefix='vh_', size='vh_size')
  tree.define_collection(name='tracks', prefix='vt_', size='vt_size')
  tree.define_collection(name='particles', prefix='vp_', size='vp_size')
  return tree

In [4]:
maxEvents = 20000

debug = 0

kDT, kCSC, kRPC, kGEM, kME0 = 0, 1, 2, 3, 4

def find_endsec(endcap, sector):
  endsec = (sector - 1) if endcap == 1 else (sector - 1 + 6)
  return endsec

def analysis(strip_unit=8*4, verbose=1):
  tree = load_pgun()
  
  outs = []

  # Loop over events
  for ievt, evt in enumerate(tree):
    if maxEvents != -1 and ievt == maxEvents:
      break
    
    if verbose and (ievt % 1000 == 0):  print("Processing event: {0}".format(ievt))
    
    # Skip events with very few hits
    if not len(evt.hits) >= 4:
      continue
    
    # Skip events without ME1 hits
    has_ME1 = False
    for ihit, hit in enumerate(evt.hits):
      if hit.type == kCSC and hit.station == 1:
        has_ME1 = True
        break
    if not has_ME1:
      continue
    
    part = evt.particles[0]  # particle gun
    part.invpt = np.true_divide(part.q, part.pt)
    
    # Find the best sector
    sector_cnt_array = np.zeros((12,), dtype=np.int32)
    sector_phi_array = np.empty((12,), dtype=np.object)
    for ind in np.ndindex(sector_phi_array.shape):
      sector_phi_array[ind] = []
    
    for ihit, hit in enumerate(evt.hits):
      #print(".. hit  {0} {1} {2} {3} {4} {5} {6} {7} {8} {9} {10} {11}".format(ihit, hit.bx, hit.type, hit.station, hit.ring, hit.sector, hit.fr, hit.sim_phi, hit.sim_theta, hit.time, hit.sim_tp1, hit.sim_tp2))
      
      endsec = find_endsec(hit.endcap, hit.sector)
      assert(hit.emtf_phi < 5040)  # 84*60
      
      sector_cnt_array[endsec] += 1
      sector_phi_array[endsec].append(hit.emtf_phi)
    
    best_sector = np.argmax(sector_cnt_array)
    #print "ievt {0} nhits {1}".format(ievt, len(evt.hits))
    #print "..", sector_cnt_array
    #print "..", sector_cnt_array[best_sector]
    #print "..", sector_phi_array[best_sector]
    
    # Find the best phi bin
    phis = sector_phi_array[best_sector]
    phis = np.asarray(phis)
    
    # Overlapping bins
    bins1 = np.arange(0, 5120, strip_unit)
    bins2 = np.arange(-strip_unit/2, 5120+strip_unit/2, strip_unit)
    if len(bins2):
      binned_phis1 = np.digitize(phis, bins=bins1[1:])  # skip lowest edge
      binned_phis2 = np.digitize(phis, bins=bins2[1:])  # skip lowest edge
      binned_phis_with_counts1 = np.unique(binned_phis1, return_counts=True)
      binned_phis_with_counts2 = np.unique(binned_phis2, return_counts=True)
      if np.max(binned_phis_with_counts1[1]) >= np.max(binned_phis_with_counts2[1]):
        bins = bins1
        binned_phis = binned_phis1
        binned_phis_with_counts = binned_phis_with_counts1
      else:
        bins = bins2
        binned_phis = binned_phis2
        binned_phis_with_counts = binned_phis_with_counts2
    else:
      bins = bins1
      binned_phis = np.digitize(phis, bins=bins[1:])  # skip lowest edge
      binned_phis_with_counts = np.unique(binned_phis, return_counts=True)
    
    best_bin_tmp = np.argmax(binned_phis_with_counts[1])
    best_bin = binned_phis_with_counts[0][best_bin_tmp]
    best_bin_cnt = binned_phis_with_counts[1][best_bin_tmp]
    #frac = float(best_bin_cnt)/len(phis)
    
    best_bin_phis = phis[binned_phis == best_bin]
    if len(best_bin_phis):
      delta = np.max(best_bin_phis) - np.min(best_bin_phis)
    else:
      delta = 0
    
    # Debug
    if verbose and debug:
      print part.pt, best_bin, len(phis), best_bin_cnt, delta, phis, binned_phis_with_counts1, binned_phis_with_counts2
    
    outs.append((part.pt, len(phis), best_bin_cnt, delta))

  return np.asarray(outs)

In [5]:
#eff_pt_bins = (0., 0.5, 1., 1.5, 2., 3., 4., 5., 6., 7., 8., 10., 12., 14., 16., 18., 20., 22., 24., 27., 30., 34., 40., 48., 60., 80., 120.)
eff_pt_bins = (0., 0.5, 1., 1.5, 2., 3., 4., 5., 6., 7., 8., 10., 12., 14., 16., 18., 20., 22., 24., 27., 30.)
edges = np.asarray(eff_pt_bins)

def post_analysis(outs):
  xdata = (edges[1:] + edges[:-1])/2

  denom = np.zeros_like(xdata, dtype=np.float32)
  numer = np.zeros_like(xdata, dtype=np.float32)

  for o in outs:
    (pt, cnt, bin_cnt, delta) = o
    x = np.digitize(pt, bins=eff_pt_bins[1:])  # skip lowest edge
    if x == len(xdata):
      x -= 1
    denom[x] += 1
    if float(bin_cnt)/cnt > 7./8:
      numer[x] += 1

  denom[denom == 0] = 1
  ydata = np.true_divide(numer, denom)
  return xdata, ydata

In [6]:
# Run analysis

strip_unit = 5120/8
outs = analysis(strip_unit=strip_unit)
xdata, ydata = post_analysis(outs)

print strip_unit
print outs
print xdata, ydata

[INFO] Opening file: ../test7/ntuple_SingleMuon_Toy_2GeV_add.6.root
Processing event: 0
Processing event: 1000
Processing event: 2000
Processing event: 3000
Processing event: 4000
Processing event: 5000
Processing event: 6000
Processing event: 7000
Processing event: 8000
Processing event: 9000
Processing event: 10000
Processing event: 11000
Processing event: 12000
Processing event: 13000
Processing event: 14000
Processing event: 15000
Processing event: 16000
Processing event: 17000
Processing event: 18000
Processing event: 19000
640
[[  4.08118868   7.           7.         299.        ]
 [  2.75273561   8.           8.         206.        ]
 [  2.30978417  10.          10.         509.        ]
 ...
 [  3.34204245   6.           5.          72.        ]
 [  2.00479174   6.           6.         170.        ]
 [  8.06097412  11.          11.         126.        ]]
[ 0.25  0.75  1.25  1.75  2.5   3.5   4.5   5.5   6.5   7.5   9.   11.
 13.   15.   17.   19.   21.   23.   25.5  28.5 ] [0. 

In [7]:
# Run more analysis

if False:
  plt.figure()

  strip_units = [8*4, 8*8, 8*16, 8*32, 5120/16, 5120/8, 5120/4, 5120/2]

  for strip_unit in strip_units:
    print('Using strip_unit {0}'.format(strip_unit))
    outs = analysis(strip_unit=strip_unit, verbose=0)
    xdata, ydata = post_analysis(outs)
    xerr = (edges[1:] - edges[:-1])/2
    #plt.errorbar(xdata, ydata, xerr=xerr, marker=',', fmt='o', lw=1)
    plt.errorbar(xdata, ydata, xerr=xerr, marker=',', lw=1)

  leg = plt.legend(strip_units, frameon=True, framealpha=0.9, fontsize=10)
  leg.get_frame().set_facecolor('#ededed')
  plt.xlim(0,35)
  plt.ylim(0,1.1)
  plt.xlabel(r'gen $p_{T}$ [GeV]')
  plt.ylabel(r'$\varepsilon$')
  plt.show()

In [8]:
if False:
  # Mock minbias pT spectrum
  #xx = np.linspace(0.1, 49.1, num=50)
  xx = np.linspace(2.1, 52.1, num=101)
  #reweight = lambda x: 5.5 * np.power(x,-3)
  #reweight = lambda x: 11 * np.power(x,-4)
  reweight = lambda x: 7.778 * np.power(x,-3.5)
  xw = np.fromiter((reweight(xi) for xi in xx), xx.dtype)
  xw_cumsum = np.cumsum(xw)
  xw_cumsum /= xw.sum()
  print zip(xx, xw_cumsum)

In [ ]:
maxEvents = 20000
#maxEvents = 200000

debug = 0

kDT, kCSC, kRPC, kGEM, kME0 = 0, 1, 2, 3, 4

nlayers = 12  # 5 (CSC) + 4 (RPC) + 3 (GEM)


# Make EMTF image
# A m-by-n matrix has m rows and n columns
class EMTFImage(object):
  def __init__(self):
    l_lut = np.zeros((5,5,5), dtype=np.int32) - 99  # type, station, ring -> layer
    l_lut[1,1,4] = 0  # ME1/1a
    l_lut[1,1,1] = 0  # ME1/1b
    l_lut[1,1,2] = 1  # ME1/2
    l_lut[1,1,3] = 1  # ME1/3
    l_lut[1,2,1] = 2  # ME2/1
    l_lut[1,2,2] = 2  # ME2/2
    l_lut[1,3,1] = 3  # ME3/1
    l_lut[1,3,2] = 3  # ME3/2
    l_lut[1,4,1] = 4  # ME4/1
    l_lut[1,4,2] = 4  # ME4/2
    l_lut[2,1,2] = 1  # RE1/2
    l_lut[2,2,2] = 2  # RE2/2
    l_lut[2,3,1] = 3  # RE3/1
    l_lut[2,3,2] = 3  # RE3/2
    l_lut[2,3,3] = 3  # RE3/3
    l_lut[2,4,1] = 4  # RE4/1
    l_lut[2,4,2] = 4  # RE4/2
    l_lut[2,4,3] = 4  # RE4/3
    l_lut[3,1,1] = 0  # GE1/1
    l_lut[3,2,1] = 2  # GE2/1
    l_lut[4,1,1] = 0  # ME0
    self.l_lut = l_lut
    
    m_lut = np.zeros((12,), dtype=np.int32) - 99  # layer -> row
    m_lut[0]  = 0
    m_lut[1]  = 1
    m_lut[2]  = 2
    m_lut[3]  = 3
    m_lut[4]  = 4
    m_lut[5]  = 1
    m_lut[6]  = 2
    m_lut[7]  = 3
    m_lut[8]  = 4
    m_lut[9]  = 0
    m_lut[10] = 2
    m_lut[11] = 0
    self.m_lut = m_lut
    
    self.superstrip_size = 16
    
    # ME1/1, ME1/2, ME2, ME3, ME4, RE1
    # RE2, RE3, RE4, GE1/1, GE2/1, ME0
    s_bend_lut =[-0.281694, -0.165681, -0.630045,  0.375211,  0.435738,  1.000000,
                  1.000000,  1.000000,  1.000000, -0.523649, -0.731835, -0.111334,]
    self.s_bend_lut = np.array(s_bend_lut, dtype=np.float32)

  def get_layer(self, hit):
    index = (hit.type, hit.station, hit.ring)
    lay = self.l_lut[index]
    return lay
  
  def get_zone(self, hit):
    zone_word = 0
    if 0 <= hit.emtf_theta <= 18:
      zone_word |= (1 << 0)
    if 14 <= hit.emtf_theta <= 25:
      zone_word |= (1 << 1)
    if 20 <= hit.emtf_theta <= 36:
      zone_word |= (1 << 2)
    if 30 <= hit.emtf_theta <= 51:
      zone_word |= (1 << 3)
    if 44 <= hit.emtf_theta <= 68:
      zone_word |= (1 << 4)
    if 56 <= hit.emtf_theta <= 88:
      zone_word |= (1 << 5)
    return zone_word
    
  def get_row(self, hit):
    lay = self.get_layer(hit)
    m = self.m_lut[lay]
    return m
  
  def get_col(self, hit):
    f = lambda x : x//self.superstrip_size
    n = f(hit.emtf_phi)
    return n
  
  def get_chn_is_csc_f(self, hit):
    return hit.type == kCSC and hit.fr == 1

  def get_chn_is_csc_r(self, hit):
    return hit.type == kCSC and hit.fr == 0
    
  def get_chn_is_rpc(self, hit):
    return hit.type == kRPC

  def get_chn_is_gem(self, hit):
    return hit.type == kGEM

  def get_chn_is_me0(self, hit):
    return hit.type == kME0
  
  def get_chn_bend(self, hit):
    if hit.type == kCSC:
      bend = hit.bend
      bend *= hit.endcap
    elif hit.type == kGEM:
      bend = hit.bend
      bend *= hit.endcap
    else:
      bend = hit.bend
    lay = self.get_layer(hit)
    s = self.s_bend_lut[lay]
    bend *= s
    return bend
  
  def get_chn_theta(self, hit):
    f = lambda x : (x - 3.)/83.
    theta = f(hit.emtf_theta)
    return theta
  
  def get_chn_mphi(self, hit):
    f = lambda x : (x % self.superstrip_size)/float(self.superstrip_size)
    mphi = f(hit.emtf_phi)
    return mphi
  
  def get_chn(self, hit):
    chn = (self.get_chn_is_csc_f(hit),
           self.get_chn_is_csc_r(hit),
           self.get_chn_is_rpc(hit),
           self.get_chn_is_gem(hit),
           self.get_chn_is_me0(hit),
           self.get_chn_bend(hit),
           self.get_chn_theta(hit),
           self.get_chn_mphi(hit))
    return chn
  
  def __call__(self, hits):
    amap = {}
    for ihit, hit in enumerate(evt.hits):
      m = get_row(hit)
      n = get_col(hit)
      amap.setdefault((m,n), []).append(hit)
      
    alist = []
    
    priority = (4, 0, 3, 2, 1)  # CSC (0) > ME0 (1) > GEM (2) > RPC (3) > DT (4)
    for k, v in amap.iteritems():
      (m,n) = k
      hits_mn = v
      hits_mn.sort(key=lambda x: priority[x.type])
      h = hits_mn[0]
      chn = self.get_chn(h)
      alist.append((m,n) + chn)


#FIXME: add eta partitions
#FIXME: format 1/pT
#FIXME: add other variables
#FIXME: dont reduce strip_unit by 8
def make_images(strip_unit=8*4, verbose=1):
  tree = load_pgun()
  
  images = []
  labels = []

  # Loop over events
  for ievt, evt in enumerate(tree):
    if maxEvents != -1 and ievt == maxEvents:
      break
    
    if verbose and (ievt % 1000 == 0):  print("Processing event: {0}".format(ievt))
    
    # Skip events with very few hits
    if not len(evt.hits) >= 4:
      continue
    
    # Skip events without ME1 hits
    has_ME1 = False
    for ihit, hit in enumerate(evt.hits):
      if hit.type == kCSC and hit.station == 1:
        has_ME1 = True
        break
    if not has_ME1:
      continue
    
    part = evt.particles[0]  # particle gun
    part.invpt = np.true_divide(part.q, part.pt)
    
    # Find the best sector
    sector_cnt_array = np.zeros((12,), dtype=np.int32)
    sector_hits_array = np.empty((12,), dtype=np.object)
    for ind in np.ndindex(sector_hits_array.shape):
      sector_hits_array[ind] = []
    
    for ihit, hit in enumerate(evt.hits):
      #print(".. hit  {0} {1} {2} {3} {4} {5} {6} {7} {8} {9} {10} {11}".format(ihit, hit.bx, hit.type, hit.station, hit.ring, hit.sector, hit.fr, hit.sim_phi, hit.sim_theta, hit.time, hit.sim_tp1, hit.sim_tp2))
      
      endsec = find_endsec(hit.endcap, hit.sector)
      
      sector_cnt_array[endsec] += 1
      sector_hits_array[endsec].append(hit)
    
    best_sector = np.argmax(sector_cnt_array)
    
    # Find the best superstrip, include overlaps
    sector_hits = sector_hits_array[best_sector]
    phis = [hit.emtf_phi for hit in sector_hits]
    bins1 = np.arange(0, 5120, strip_unit)
    bins2 = np.arange(-strip_unit/2, 5120+strip_unit/2, strip_unit)
    if len(bins2):
      binned_phis1 = np.digitize(phis, bins=bins1[1:])  # skip lowest edge
      binned_phis2 = np.digitize(phis, bins=bins2[1:])  # skip lowest edge
      binned_phis_with_counts1 = np.unique(binned_phis1, return_counts=True)
      binned_phis_with_counts2 = np.unique(binned_phis2, return_counts=True)
      if np.max(binned_phis_with_counts1[1]) >= np.max(binned_phis_with_counts2[1]):
        bins = bins1
        binned_phis = binned_phis1
        binned_phis_with_counts = binned_phis_with_counts1
      else:
        bins = bins2
        binned_phis = binned_phis2
        binned_phis_with_counts = binned_phis_with_counts2
    else:
      bins = bins1
      binned_phis = np.digitize(phis, bins=bins[1:])  # skip lowest edge
      binned_phis_with_counts = np.unique(binned_phis, return_counts=True)
    
    best_bin_tmp = np.argmax(binned_phis_with_counts[1])
    best_bin = binned_phis_with_counts[0][best_bin_tmp]
    best_bin_cnt = binned_phis_with_counts[1][best_bin_tmp]
    #frac = float(best_bin_cnt)/len(phis)
    
    best_bin_idx = (binned_phis == best_bin)
    reduce_list_f = lambda x, mask: [x[i] for i in np.where(mask)[0]]
    best_bin_hits = reduce_list_f(sector_hits, best_bin_idx)
    
    #image_shape = (nlayers, strip_unit, nchannels)
    #image_shape = (nlayers, strip_unit)
    image_shape = (nlayers, strip_unit/8)
    image = np.zeros(image_shape, dtype=np.float32)
    
    # Debug
    if verbose and debug and ievt < 10:
      print("ievt {0}".format(ievt))
    
    for ihit, hit in enumerate(best_bin_hits):
      l = find_emtf_layer(hit)
      c = find_emtf_channel(hit)
      phi = hit.emtf_phi
      theta = hit.emtf_theta
      time = hit.bx
      
      phi_sub = phi - bins[best_bin]
      assert(0 <= phi_sub < strip_unit)
      
      # reorder layers
      ll = ordered_layer_mapping[l]
      
      #FIXME: ignore channels for now
      #image[ll, phi_sub, c] = 1
      #image[ll, phi_sub] = 1
      image[ll, phi_sub/8] = 1
      
      # Debug
      if verbose and debug and ievt < 10:
        print "..", (l, c, phi, theta, time), bins[best_bin], phi_sub
    
    images.append(image)
    
    label = (part.pt > 14)
    labels.append(label)
  
  return np.asarray(images), np.asarray(labels)

In [ ]:
strip_unit = 5120/8
images, labels = make_images(strip_unit=strip_unit)
print images.shape, labels.shape

outfile = 'histos_tbe.npz'
np.savez_compressed(outfile, images=images, labels=labels)

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (5,2.5)
#mpl.rcParams['axes.labelpad'] = 0
#mpl.rcParams['axes.labelsize'] = 0
#mpl.rcParams['xtick.labelsize'] = 0
#mpl.rcParams['ytick.labelsize'] = 0

In [ ]:
#aspect = float(strip_unit)/nlayers
#aspect = 20
aspect = 'auto'

image = images[0]
print image.shape
#print np.where(image[:,:,0])
#print np.where(image[:,:,1])
#print np.where(image[:,:,2])
#print np.where(image[:,:,3])
#plt.imshow(image[:,:,0], cmap='viridis', interpolation='none', extent=(0,strip_unit,nlayers,0), aspect=aspect)
#plt.show()
#plt.imshow(image[:,:,1], cmap='viridis', interpolation='none', extent=(0,strip_unit,nlayers,0), aspect=aspect)
#plt.show()
#plt.imshow(image[:,:,2], cmap='viridis', interpolation='none', extent=(0,strip_unit,nlayers,0), aspect=aspect)
#plt.show()
#plt.imshow(image[:,:,3], cmap='viridis', interpolation='none', extent=(0,strip_unit,nlayers,0), aspect=aspect)
#plt.show()
plt.imshow(image, cmap='viridis', interpolation='none', extent=(0,strip_unit/8,nlayers,0), aspect=aspect)
plt.show()

In [ ]:
image = images[1]
#print np.where(image[:,:,0])
#plt.imshow(image[:,:,0], cmap='viridis', interpolation='none', extent=(0,strip_unit,nlayers,0), aspect=aspect)
plt.imshow(image, cmap='viridis', interpolation='none', extent=(0,strip_unit/8,nlayers,0), aspect=aspect)
plt.show()

In [ ]:
image = images[2]
#print np.where(image[:,:,0])
#plt.imshow(image[:,:,0], cmap='viridis', interpolation='none', extent=(0,strip_unit,nlayers,0), aspect=aspect)
plt.imshow(image, cmap='viridis', interpolation='none', extent=(0,strip_unit/8,nlayers,0), aspect=aspect)
plt.show()

In [ ]:
image = images[3]
#print np.where(image[:,:,0])
#plt.imshow(image[:,:,0], cmap='viridis', interpolation='none', extent=(0,strip_unit,nlayers,0), aspect=aspect)
plt.imshow(image, cmap='viridis', interpolation='none', extent=(0,strip_unit/8,nlayers,0), aspect=aspect)
plt.show()